In [8]:
using CSV

In [9]:
root = dirname(@__FILE__)
inputAIS = joinpath(root, "AI_factor_full_1997-2021.txt")
inputISI = joinpath(root, "IF_factor_full_1997-2021.txt")

"/home/lali/TITAN-ROG-sync/julia/ISI/IF_factor_full_1997-2021.txt"

In [10]:
AIS = CSV.read(inputAIS, NamedTuple; header=true)
AIS[8157, [1, 2, 3, 14, 25, 26]]

(Journal = String31["ANNU REV BIOCHEM", "NAT GENET", "ANNU REV IMMUNOL", "CELL", "NAT MED", "NEW ENGL J MED", "NATURE", "SCIENCE", "ENDOCR REV", "ANNU REV NEUROSCI"  …  "SMITH COLL STUD SOC", "HIST ECON IDEAS", "INTERLEND DOC SUPPLY", "MIL LAW REV", "REV INT PSYCHOL SOC", "RECH ECON LOUVAIN", "J E ASIA INT LAW", "TRANSYLV REV", "ROAD TRANSP RES", "POLICY REV"], 1997 = [20.985, 13.779, 26.653, 18.188, 12.745, 17.864, 16.996, 16.539, 9.23, 17.191  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1998 = [20.985, 13.779, 26.653, 18.188, 12.745, 17.864, 16.996, 16.539, 9.23, 17.191  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1999 = [20.985, 13.779, 26.653, 18.188, 12.745, 17.864, 16.996, 16.539, 9.23, 17.191  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 2000 = [20.985, 13.779, 26.653, 18.188, 12.745, 17.864, 16.996, 16.539, 9.23, 17.191  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 2001 = [20.985, 13.779, 26.653, 18.188, 12.745, 17.864, 16.996, 16.539, 9.23

In [ ]:
ISI = CSV.read(inputISI, DataFrame; header=true)
ISI[8157, [1, 2, 3, 14, 25, 26]]

In [ ]:
root = dirname(@__FILE__)
inputMINE = joinpath(root, "LS_savedrecs_mine.txt")
inputCIT = joinpath(root, "LS_savedrecs.txt")

In [ ]:
MINEraw = CSV.read(inputMINE, DataFrame; header=true)
eltype.(eachcol(MINEraw));

In [ ]:
MINE = MINEraw[:, [2, 9, 24, 30, 32, 42, 45, 46, 47, 52, 53, 54, 55]];
eltype.(eachcol(MINE));

In [ ]:
MINE[1, [:PY]] .= 2021   # ADV MAT no year!!!!
eltype.(eachcol(MINE));

In [ ]:
#root = dirname("/mnt/d/Stoleriu/")
#outputMINE = joinpath(root, "MINE_test.csv")
#CSV.write(outputMINE, MINE)

Pregatim MINE cu noi coloane

In [ ]:
rename!(MINE,:"J9" => :"Journal")
insertcols!(MINE, :NA => 0);
insertcols!(MINE, :PRIM => 0);
insertcols!(MINE, :AI => 0.0);
insertcols!(MINE, :IF => 0.0);

Gasim numarul de autori si prim autor

In [ ]:
for (row_index, lucrare) in enumerate(eachrow(MINE))
    MINE[row_index, [:NA]] .= (1+count(";", lucrare[:AU]))
    if occursin("Stoleriu", lucrare[:RP])
        MINE[row_index, [:PRIM]] .= 1
    end
    #println(row_index, MINE[row_index, [:NA]])
    #println(row_index, MINE[row_index, [:PRIM]])
end

In [ ]:
#MINE[:, [6, 7, 8, 12, 14, 15, 16, 17]]
MINE.PY = Int64.(MINE.PY)
MINE[ismissing.(MINE[!, :PY]), [1, 6, 7, 8, 12, 14, 15, 16, 17]]

In [ ]:
#root = dirname("/mnt/d/Stoleriu/")
#outputMINE = joinpath(root, "MINE_test.csv")
#CSV.write(outputMINE, MINE)

In [ ]:
AISsubsect = filter(r -> any(occursin.(Vector(MINE[1, [:Journal]]), r.Journal)), AIS)
#subsect2 = filter(r -> (["ADV MATER"][1] .== r.Journal[1]), AIS)
size(subsect, 1)
#["ADV MATER"][1] .== subsect[1, [:Journal]][1]

In [ ]:
for (row_index, lucrare) in enumerate(eachrow(MINE))
    #lucrare = MINE[1, :]
    if(ismissing(lucrare[:Journal]))
        continue
    end
    for AISrow in eachrow(AIS)
        if( lucrare[:Journal] .== AISrow[:Journal] )
            anString = string(lucrare[:PY])
            #println(AISrow[anString])
            MINE[row_index, [:AI]] .= AISrow[anString]
        end
    end
    for ISIrow in eachrow(ISI)
        if( lucrare[:Journal] .== ISIrow[:Journal] )
            anString = string(lucrare[:PY])
            #println(ISIrow[anString])
            MINE[row_index, [:IF]] .= ISIrow[anString]
        end
    end
end

In [ ]:
root = dirname("/mnt/d/Stoleriu/")
outputMINE = joinpath(root, "MINE_test.csv")
CSV.write(outputMINE, MINE)